In [1]:
import h5py
import numpy as np
import pandas as pd
import pickle
from sklearn.utils import shuffle
from keras.utils import np_utils

from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop
import xgboost as xgb
from sklearn import svm

tmp_dir = 'cache/'
np.random.seed = 2017
SEED = 2017

Using TensorFlow backend.
/home/maqiao/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# 载入特征向量
def load_data():
    X_train = []
    X_test = []
    for filename in ['gap_ResNet50.h5', 'gap_InceptionV3.h5', 'gap_Xception.h5']:
        with h5py.File(tmp_dir + filename, 'r') as h:
            X_train.append(np.array(h['train']))
            X_test.append(np.array(h['test']))
            y_train = np.array(h['label'])
    X_train = np.concatenate(X_train, axis=1)
    X_test = np.concatenate(X_test, axis=1)
    return(X_train, y_train, X_test)

In [3]:
def load_info():
    with open(tmp_dir + 'data_info.pkl', 'rb') as f:
        data_info = pickle.load(f)
    train_info = data_info['train']
    test_info = data_info['test']
    label_dict = data_info['label']
    label_dict_reverse = {value: key for key, value in label_dict.items()}
    return(train_info, test_info, label_dict, label_dict_reverse)

In [4]:
# 数据预处理函数，对神经网络和xgboost分别将whether_to_categorical设置为True和False
def preproc(X_train, train_df, label_dict, whether_to_categorical=True):
    df = train_df.copy()
    df['label'] = train_df['label'].map(label_dict)
    if whether_to_categorical:
        y_train = np_utils.to_categorical(df.label)
    else:
        y_train = df.label.values
    X_train, y_train = shuffle(X_train, y_train)
    return(X_train, y_train)

In [5]:
# 全连接层预测函数
def cnn_predict(X_train, y_train, X_test, epochs=45):
    input_tensor = Input(X_train.shape[1:])
    x = Dropout(0.7)(input_tensor)
    x = Dense(100, activation = 'sigmoid')(x)
    model = Model(input_tensor, x)
    model.compile(optimizer=RMSprop(lr = 1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=256, epochs=epochs, validation_split=0.2)
    preds = model.predict(X_test, verbose=1)
    return(preds)

In [6]:
# xgboost预测函数，参数待优化
def xgb_predict(X_train, y_train, X_test):
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=SEED)
    dtrain = xgb.DMatrix(X_train, label = y_train)
    dtest = xgb.DMatrix(X_val, label = y_val)
    param = {'learning_rate': 0.1, 'n_estimators': 300, 'max_depth': 5,
             'min_child_weight': 5, 'gamma': 1, 'subsample': 0.8, 'colsample_bytree': 0.8,
             'scale_pos_weight': 1, 'eta': 0.05, 'silent': 1, 'objective': 'multi:softprob', 'num_class': 100}
    num_round = 100
    plst = list(param.items())
    evallist = [(dtrain, 'train'), (dtest, 'eval')]
    bst = xgb.train(plst, dtrain, num_round, evallist)
        
    dtest = xgb.DMatrix(X_test)
    preds = bst.predict(dtest).reshape(X_test.shape[0], 100)
    return(preds)

In [7]:
def svm_predict(X_train, y_train, X_test):
    clf = svm.SVC(decision_function_shape='ovr')
    clf.fit(X_train, y_train)
    preds = clf.decision_function(X_test)
    return(preds)

In [8]:
# 对预测结果进行后处理，将label映射回134类
def postproc(preds, label_dict):
    preds = np.argmax(preds, axis=1)
    preds = pd.Series(preds)
    label_dict_reverse = {value: key for key, value in label_dict.items()}
    preds = preds.map(label_dict_reverse)
    return(preds.values)

In [9]:
# 移除数据函数
def remove_ids(X_train, train_df, rm_ids):
    """移除id在rm_ids列表中的数据"""
    remains = (1 - train_df.id.isin(rm_ids)).astype(bool).values
    X_remains = X_train[remains]
    remains_df = train_df[remains]
    return(X_remains, remains_df)

# label修改函数
def modifi_labels(train_df, modifi_df):
    """对train_df中的部分label进行修改，以modifi_df中的label为准"""
    df = pd.merge(train_df, modifi_df, on='id', how='inner')
    df = df[df.label_x != df.label_y]
    train = train_df.set_index(['id'])
    for i, row in df.iterrows():
        train.loc[row.id, 'label'] = row.label_y
    train = train.reset_index()
    return(train)

In [10]:
# 对repeated_df中重复的数据进行预测，将预测结果存入prediction_of_repeated.pkl文件
# 若预测值与之前的已有的两个label都不相同，删除之；若与其中一个相同，修改为预测值
def modifi_repeated(X_data, data_info, label_dict, epochs = 45):
    dump_file = tmp_dir + 'prediction_of_repeated.pkl'
    if os.path.exists(dump_file):
        with open(dump_file, 'rb') as f:
            pred_df = pickle.load(f)
    else:
        repeated_file = 'cache/repeated_df.pkl'
        with open(repeated_file, 'rb') as f:
            repeated = pickle.load(f)
        remains = data_info.id.isin(repeated.id.values)
        X_test = X_data[remains]
        test_info = data_info[remains]
        X_train, train_info = remove_ids(X_data, data_info, repeated.id.values)
        X_train, y_train = preproc(X_train, train_info, label_dict)
        y_preds = cnn_predict(X_train, y_train, X_test, epochs)
        test_info['label'] = postproc(y_preds, label_dict)
        pred_df = pd.merge(test_info, repeated, on='id', how='left')
        with open(dump_file, 'wb') as f:
            pickle.dump(pred_df, f)
    
    rm_ids = pred_df[(pred_df.label != pred_df.label_x) & (pred_df.label != pred_df.label_y)]['id']
    modifi_df = pred_df[(pred_df.label == pred_df.label_x) | (pred_df.label == pred_df.label_y)]
    X_data, data_info = remove_ids(X_data, data_info, rm_ids.values)
    data_info = modifi_labels(data_info, modifi_df[['id', 'label']])
    return(X_data, data_info)

In [11]:
# 对所有数据进行交叉预测，返回预测概率
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
def cross_predict(n_splits = 10, epochs=70):
    dump_file = tmp_dir + 'cross_predict.pkl'
    if os.path.exists(dump_file):
        with open(dump_file, 'rb') as f:
            df = pickle.load(f)
    else:
        train_info, test_info, label_dict, label_dict_reverse = load_info()
        X_train, y_train, X_test = load_data()
        X, y_df = shuffle(X_train, train_info)
        kf = KFold(n_splits = n_splits)
        y_preds = []
        for train, test in kf.split(X):
            print(u'\n第%d次交叉预测' % (len(y_preds) + 1))
            X_train, X_test, y_train_df, y_test_df = X[train], X[test], y_df.iloc[train,:], y_df.iloc[test,:]
            X_train, y_train = preproc(X_train, y_train_df, label_dict)
            y_pred = cnn_predict(X_train, y_train, X_test, epochs)
            y_preds.append(y_pred)
        y_preds = np.concatenate(y_preds)
    
        preds = []
        max_probas = []
        for ys in y_preds:
            pred = np.argmax(ys)
            total = sum(ys)
            ys = ys / total
            max_proba = max(ys)
            preds.append(pred)
            max_probas.append(max_proba)
        df = y_df.copy()
        df['pred'] = preds
        df['max_proba'] = max_probas
        df['pred'] = df['pred'].map(label_dict_reverse).astype(int)
    
        wrong_rate = df[df.label != df.pred].shape[0] / df.shape[0]
        print(u'%d折交叉预测错误率: %f' % (n_splits, wrong_rate))
        with open(dump_file, 'wb') as f:
            pickle.dump(df, f)
    return(df)

In [12]:
# 依据cross_predict函数预测的概率
# 若概率大于modifi_point，说明该数据可能标注错误，修改为预测的lable
# 若概率小于rm_point, 可能不是狗的图片或狗的特征不明显的图片，删除它们
def remove_and_modifi(X_train, train_info, modifi_point = 0.9, rm_point = 0.5):
    cv_df = cross_predict()
    modifi_df = cv_df[(cv_df.label != cv_df.pred) & (cv_df.max_proba > modifi_point)]
    rm_ids = cv_df[(cv_df.label != cv_df.pred) & (cv_df.max_proba < rm_point)]['id']
    print(u'修改阈值为%f, 修改数据%d/%d个' % (modifi_point, modifi_df.shape[0], X_train.shape[0]))
    print(u'删除阈值为%f, 删除数据%d/%d个' % (rm_point, rm_ids.shape[0], X_train.shape[0]))
    modifi_df = modifi_df[['id', 'pred']]
    modifi_df.columns = ['id', 'label']
    train_info = modifi_labels(train_info, modifi_df)
    X_train, train_info = remove_ids(X_train, train_info, rm_ids)
    
    return(X_train, train_info)

In [13]:
# 测试函数，使用remove_and_modifi函数对数据进行删除和修改label后，测试其泛化表现
def remove_and_modifi_val(model, X_train, train_info, label_dict, modifi_point = 1, rm_point = 0):
    X_train, X_val, train_info, val_info = train_test_split(X_train, train_info, test_size=0.3, random_state=SEED)
    if (modifi_point < 1) or (rm_point > 0):
        X_train, train_info = remove_and_modifi(X_train, train_info, modifi_point, rm_point)
    if model == 'cnn':
        X_train, y_train = preproc(X_train, train_info, label_dict, True)
        y_preds = cnn_predict(X_train, y_train, X_val, 70)
    elif model == 'xgb':
        X_train, y_train = preproc(X_train, train_info, label_dict, False)
        y_preds = xgb_predict(X_train, y_train, X_val)
    val_info['pred'] = postproc(y_preds, label_dict)
    wrong_num = val_info[val_info.label != val_info.pred].shape[0]
    print(u'\n在验证集上错误率为%f' % (wrong_num / val_info.shape[0]))

In [20]:
train_info, test_info, label_dict, label_dict_reverse = load_info()
X_train, y_train, X_test = load_data()
print(u'修改重复数据前有数据%d个' % X_train.shape[0])
X_train, train_info = modifi_repeated(X_train, train_info, label_dict, 70)
print(u'修改重复数据后有数据%d个' % X_train.shape[0])
X_train, train_info = shuffle(X_train, train_info)

修改重复数据前有数据18686个
修改重复数据后有数据18673个


In [15]:
# 不断调整modifi_point和rm_point的取值进行交叉训练，
# 得出在modifi_point=0.88, rm_point=0.3时，线下得分最高
remove_and_modifi_val('cnn', X_train, train_info, label_dict, 0.88, 0.35)

修改阈值为0.880000, 修改数据754/13071个
删除阈值为0.350000, 删除数据200/13071个
Train on 10346 samples, validate on 2587 samples
Epoch 1/70
10346/10346 [==============================] - 0s - loss: 4.3452 - acc: 0.0536 - val_loss: 3.5120 - val_acc: 0.3533
Epoch 2/70
10346/10346 [==============================] - 0s - loss: 3.4993 - acc: 0.1997 - val_loss: 2.4737 - val_acc: 0.5991
Epoch 3/70
10346/10346 [==============================] - 0s - loss: 2.6653 - acc: 0.3712 - val_loss: 1.7523 - val_acc: 0.6966
Epoch 4/70
10346/10346 [==============================] - 0s - loss: 2.0580 - acc: 0.4989 - val_loss: 1.3247 - val_acc: 0.7426
Epoch 5/70
10346/10346 [==============================] - 0s - loss: 1.6356 - acc: 0.5798 - val_loss: 1.0732 - val_acc: 0.7630
Epoch 6/70
10346/10346 [==============================] - 0s - loss: 1.3682 - acc: 0.6347 - val_loss: 0.9150 - val_acc: 0.7886
Epoch 7/70
10346/10346 [==============================] - 0s - loss: 1.1833 - acc: 0.6764 - val_loss: 0.8136 - val_acc: 0.7963
Ep

10346/10346 [==============================] - 0s - loss: 0.2472 - acc: 0.9189 - val_loss: 0.4532 - val_acc: 0.8547
Epoch 64/70
10346/10346 [==============================] - 0s - loss: 0.2477 - acc: 0.9203 - val_loss: 0.4537 - val_acc: 0.8527
Epoch 65/70
10346/10346 [==============================] - 0s - loss: 0.2482 - acc: 0.9191 - val_loss: 0.4550 - val_acc: 0.8516
Epoch 66/70
10346/10346 [==============================] - 0s - loss: 0.2377 - acc: 0.9254 - val_loss: 0.4538 - val_acc: 0.8539
Epoch 67/70
10346/10346 [==============================] - 0s - loss: 0.2393 - acc: 0.9219 - val_loss: 0.4560 - val_acc: 0.8535
Epoch 68/70
10346/10346 [==============================] - 0s - loss: 0.2338 - acc: 0.9249 - val_loss: 0.4528 - val_acc: 0.8539
Epoch 69/70
10346/10346 [==============================] - 0s - loss: 0.2277 - acc: 0.9272 - val_loss: 0.4539 - val_acc: 0.8520
Epoch 70/70
5280/5602 [===========================>..] - ETA: 0s
在验证集上错误率为0.198322


/home/maqiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [19]:
# 将提取后的特征向量，使用全连接层训练、和xgboost训练后进行融合
X_train, X_val, train_info, val_info = train_test_split(X_train, train_info, test_size=0.3, random_state=SEED)
X_train, train_info = remove_and_modifi(X_train, train_info, modifi_point = 0.88, rm_point = 0.3)

X_train1, y_train1 = preproc(X_train, train_info, label_dict, True)
X_train2, y_train2 = preproc(X_train, train_info, label_dict, False)

pred1 = cnn_predict(X_train1, y_train1, X_val, 70)
pred2 = xgb_predict(X_train2, y_train2, X_val)

pred3 = pred1.copy()
for i in range(len(pred1)):
    total = sum(pred1[i])
    ys = pred1[i] / total
    pred3[i] = ys

preds = pred3*0.55 + pred2*0.45
val_info['pred'] = postproc(preds, label_dict)
wrong_num = val_info[val_info.label != val_info.pred].shape[0]
print(u'\n在验证集上错误率为%f' % (wrong_num / val_info.shape[0]))

修改阈值为0.880000, 修改数据754/13071个
删除阈值为0.300000, 删除数据103/13071个
Train on 10399 samples, validate on 2600 samples
Epoch 1/70
10399/10399 [==============================] - 0s - loss: 4.3680 - acc: 0.0482 - val_loss: 3.5996 - val_acc: 0.3092
Epoch 2/70
10399/10399 [==============================] - 0s - loss: 3.5529 - acc: 0.1867 - val_loss: 2.5601 - val_acc: 0.5915
Epoch 3/70
10399/10399 [==============================] - 0s - loss: 2.6870 - acc: 0.3702 - val_loss: 1.8083 - val_acc: 0.6877
Epoch 4/70
10399/10399 [==============================] - 0s - loss: 2.0709 - acc: 0.5005 - val_loss: 1.3753 - val_acc: 0.7385
Epoch 5/70
10399/10399 [==============================] - 0s - loss: 1.6521 - acc: 0.5838 - val_loss: 1.1119 - val_acc: 0.7631
Epoch 6/70
10399/10399 [==============================] - 0s - loss: 1.3822 - acc: 0.6411 - val_loss: 0.9550 - val_acc: 0.7738
Epoch 7/70
10399/10399 [==============================] - 0s - loss: 1.1772 - acc: 0.6836 - val_loss: 0.8510 - val_acc: 0.7881
Ep

10399/10399 [==============================] - 0s - loss: 0.2419 - acc: 0.9218 - val_loss: 0.5111 - val_acc: 0.8473
Epoch 65/70
10399/10399 [==============================] - 0s - loss: 0.2410 - acc: 0.9220 - val_loss: 0.5090 - val_acc: 0.8473
Epoch 66/70
10399/10399 [==============================] - 0s - loss: 0.2338 - acc: 0.9249 - val_loss: 0.5085 - val_acc: 0.8462
Epoch 67/70
10399/10399 [==============================] - 0s - loss: 0.2336 - acc: 0.9257 - val_loss: 0.5104 - val_acc: 0.8473
Epoch 68/70
10399/10399 [==============================] - 0s - loss: 0.2338 - acc: 0.9262 - val_loss: 0.5119 - val_acc: 0.8465
Epoch 69/70
10399/10399 [==============================] - ETA: 0s - loss: 0.2260 - acc: 0.928 - 0s - loss: 0.2258 - acc: 0.9287 - val_loss: 0.5105 - val_acc: 0.8435
Epoch 70/70
5184/5602 [==========================>...] - ETA: 0s[0]	train-merror:0.642268	eval-merror:0.651538
[1]	train-merror:0.407627	eval-merror:0.425385
[2]	train-merror:0.304869	eval-merror:0.336667
[

In [21]:
# 将提取后的特征向量，使用全连接层训练、和xgboost训练后进行融合
X_train, train_info = remove_and_modifi(X_train, train_info, modifi_point = 0.88, rm_point = 0.3)

X_train1, y_train1 = preproc(X_train, train_info, label_dict, True)
X_train2, y_train2 = preproc(X_train, train_info, label_dict, False)

pred1 = cnn_predict(X_train1, y_train1, X_test, 70)
pred2 = xgb_predict(X_train2, y_train2, X_test)

pred3 = pred1.copy()
for i in range(len(pred1)):
    total = sum(pred1[i])
    ys = pred1[i] / total
    pred3[i] = ys

preds = pred3*0.55 + pred2*0.45
preds = postproc(preds, label_dict)
submission = pd.DataFrame({'label': preds, 'id':test_info.id})

from datetime import datetime
submission.to_csv('sub/sub_%s_3.txt' % datetime.now().strftime('%Y-%m-%d'),
                  sep='\t', columns=['label', 'id'], header=False, index=False)
submission.head(10)

修改阈值为0.880000, 修改数据754/18673个
删除阈值为0.300000, 删除数据103/18673个
Train on 14856 samples, validate on 3714 samples
Epoch 1/70
14856/14856 [==============================] - 0s - loss: 4.2405 - acc: 0.0740 - val_loss: 3.1390 - val_acc: 0.4639
Epoch 2/70
14856/14856 [==============================] - 0s - loss: 3.0408 - acc: 0.2929 - val_loss: 1.8945 - val_acc: 0.6753
Epoch 3/70
14856/14856 [==============================] - 0s - loss: 2.0966 - acc: 0.4859 - val_loss: 1.2856 - val_acc: 0.7412
Epoch 4/70
14856/14856 [==============================] - 0s - loss: 1.5549 - acc: 0.5990 - val_loss: 0.9901 - val_acc: 0.7763
Epoch 5/70
14856/14856 [==============================] - 0s - loss: 1.2585 - acc: 0.6583 - val_loss: 0.8322 - val_acc: 0.7946
Epoch 6/70
14856/14856 [==============================] - 0s - loss: 1.0720 - acc: 0.6997 - val_loss: 0.7370 - val_acc: 0.8018
Epoch 7/70
14856/14856 [==============================] - 0s - loss: 0.9439 - acc: 0.7277 - val_loss: 0.6771 - val_acc: 0.8123
Ep

14856/14856 [==============================] - 0s - loss: 0.2573 - acc: 0.9157 - val_loss: 0.4566 - val_acc: 0.8543
Epoch 64/70
14856/14856 [==============================] - 0s - loss: 0.2505 - acc: 0.9202 - val_loss: 0.4568 - val_acc: 0.8565
Epoch 65/70
14856/14856 [==============================] - 0s - loss: 0.2458 - acc: 0.9179 - val_loss: 0.4574 - val_acc: 0.8595
Epoch 66/70
14856/14856 [==============================] - 0s - loss: 0.2492 - acc: 0.9198 - val_loss: 0.4557 - val_acc: 0.8557
Epoch 67/70
14856/14856 [==============================] - 0s - loss: 0.2432 - acc: 0.9198 - val_loss: 0.4559 - val_acc: 0.8565
Epoch 68/70
14856/14856 [==============================] - 0s - loss: 0.2418 - acc: 0.9205 - val_loss: 0.4567 - val_acc: 0.8565
Epoch 69/70
14856/14856 [==============================] - 0s - loss: 0.2406 - acc: 0.9207 - val_loss: 0.4582 - val_acc: 0.8543
Epoch 70/70
10400/10593 [============================>.] - ETA: 0s[0]	train-merror:0.534041	eval-merror:0.541554
[1]

,id,label
0,"1705966597,1055292353",36
1,"2473192235,4122785580",45
2,"4188013342,2932602063",78
3,"1022790570,2912884051",26
4,"1031491946,2813127030",34
5,"2357588397,2237003912",88
6,"1443728590,2022328737",67
7,"2895133363,3781516925",68
8,"1551966137,2044502403",38
9,"1573069851,1977047833",42


In [26]:
import os
import shutil
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)
def create_wrong_classify(train_df, predict_df):
    files_dir = 'data/wrong_classify/train/'
    rmrf_mkdir(files_dir)
    label_name = pd.read_table('data/label_name.txt', header=None, sep=' ', na_filter=False)
    label_name.columns = ['name', 'label']
    label_name['name'] = label_name['name'].apply(lambda x: x.split('---')[-1])\
                                           .apply(lambda x: x.split('|')[0]).apply(lambda x: x.split('/')[0])
    label_set = set(train_df.label)
    remains = label_name.label.isin(label_set)
    label_name = label_name[remains]
    for i, x in label_name.iterrows():
        target_dir = files_dir + str(x['label']) + '_' + x['name']
        os.mkdir(target_dir)
        os.mkdir(target_dir + '/wrong')
    train_df = pd.merge(train_df, label_name, on='label', how='left')
    train_df = pd.merge(train_df, predict_df[['id', 'pred', 'max_proba']], on='id', how='left')
    for i, x in train_df.iterrows():
        target_dir = files_dir + str(x['label']) + '_' + x['name'] + '/'
        shutil.copyfile('data/alldata/' + x.id + '.jpg',
                         target_dir + x.id + '.jpg')
        if x.label != x.pred:
            shutil.move(target_dir + x.id + '.jpg',
                            target_dir + 'wrong/' + x.id + '_' + str(int(x.pred)) + '_' + '%.2e'%x.max_proba + '.jpg')

In [27]:
# create_wrong_classify函数为辅助函数，将预测错误的图片分在不同的文件夹，方便手动寻找预测错误的规律
cv_df = cross_predict()
create_wrong_classify(train_info, cv_df)